In [42]:
!pip install torch transformers
import torch 
from transformers import pipeline, AutoTokenizer

Keyring is skipped due to an exception: 'keyring.backends'


In [43]:
model_pipeline = pipeline("ner", "fine_tuned_model-11/checkpoint-9500", ignore_labels = [])
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')

In [44]:
#Constructing dataset from input file for performing inference

input_file_path = "test.txt"
dataset = []
i = 0
with open(input_file_path) as file:
    tokens = []
    for line in file.readlines():
        inp = line.split()
        if len(inp) == 0:
            dataset.append({
                "id": i,
                "tokens": tokens
            })
            i += 1
            tokens = []
        else:
            tokens.append(inp[0])
dataset.append({
                "id": i,
                "tokens": ['প্রতিক্রিয়া', 'হিসাবে', 'ডাব্লিউডাব্লিউই', 'বিপদগুলির', 'উপর', 'চাপ', 'দেওয়া', 'এবং', 'ভক্তদের', 'তাদের', 'রিংয়ে', 'দেখা', 'ক্রিয়াকলাপগুলি', 'নকল', 'করা', 'থেকে', 'বিরত', 'রাখতে', 'চেয়েছিল', 'বিজ্ঞাপনগুলি', 'প্রচার', 'করা', 'শুরু', 'করে।']
            })

In [49]:

#Constructs sentences to perform inference on based on id in the constructed dataset
def get_ids(i):
    sentence = ""
    for s in dataset[i]["tokens"]:
        sentence += (s + " ")
   
    tokenized_text = tokenizer(dataset[i]["tokens"], truncation = True, is_split_into_words = True)
    word_ids = tokenized_text.word_ids()
    return word_ids, sentence.strip()

#Performs prediction based on dataset index 
def get_pred(i):
    ids, sent = get_ids(i)
  
    res = model_pipeline(sent)

    prev = None 
    new_res = []
    for j, word_id in enumerate(ids[1:-1]):
        if word_id is None: 
            continue
        if prev != word_id:
            new_res.append(res[j]['entity'])
            prev = word_id
    return new_res  


In [50]:
#Converting from labels used by the model to the ones used in the input file
mapping = {
    "O": "O",
    "B-corporation": "B-CORP",
    "I-corporation": "I-CORP",
    "B-creative-work": "B-CW",
    "I-creative-work": "I-CW",
    "B-group": "B-GRP",
    "I-group": "I-GRP",
    "B-location": "B-LOC",
    "I-location": "I-LOC",
    "B-person": "B-PER",
    "I-person": "I-PER",
    "B-product": "B-PROD",
    "I-product": "I-PROD",
}

In [ ]:
#Writing out inferences in a file
from tqdm import tqdm
with open("output_new.txt", "w") as file:
    for i in tqdm(range(len(dataset))):
        pred = get_pred(i)
        for ans in pred:
            file.write(mapping[ans] + "\n")
        file.write("\n")
    
    

 49%|████▉     | 6499/13215 [23:45<25:08,  4.45it/s]